### installing requirements

In [1]:
!pip install openai google-search-results  taskflowai
!pip install serper



  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 843.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## import apis, etc

In [ ]:
from taskflowai import OpenaiModels
from google.colab import userdata
import os

# ✅ Set API Keys (You should remove them after testing for security reasons)
os.environ["OPENAI_API_KEY"] = "your api key"
os.environ["SERPER_API_KEY"] = "your api key"

# ✅ Fetch API Keys from Colab Secrets or Environment Variables
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = userdata.get("SERPER_API_KEY") or os.getenv("SERPER_API_KEY")

# ✅ Validate API Keys
if not OPENAI_API_KEY:
    raise ValueError("❌ Missing OPENAI_API_KEY. Set it in Colab using `userdata` or `os.environ`.")
if not SERPER_API_KEY:
    raise ValueError("❌ Missing SERPER_API_KEY. Set it in Colab using `userdata` or `os.environ`.")

print("✅ API Keys Loaded Successfully!")


class LoadModel:
    @staticmethod
    def load_openai_model():
        """Load and return the OpenAI GPT-3.5-turbo model."""
        try:
            return OpenaiModels.gpt_3_5_turbo
        except Exception as e:
            raise RuntimeError(f"Error loading OpenAI model: {e}") from e


✅ API Keys Loaded Successfully!


### Wiki_Articles Tool

In [3]:
from taskflowai import WikipediaTools

class WikiArticles:
    @staticmethod
    def fetch_articles(query: str):
        try:
            articles = WikipediaTools.search_articles(query=query)
            return articles
        except Exception as e:
            raise RuntimeError(f"Error fetching Wikipedia articles: {e}") from e


### Wiki_Images Tool

In [4]:
from taskflowai import WikipediaTools

class WikiImages:
    @staticmethod
    def search_images(query: str):
        """Search for images on Wikipedia based on the given query."""
        try:
            return WikipediaTools.search_images(query=query)
        except Exception as e:
            raise RuntimeError(f"Error fetching Wikipedia images: {e}") from e


### Serper Search  Tool

In [5]:
from taskflowai import WebTools
from typing import Optional

class SerperSearch:
    @staticmethod
    def search_web(query: str, num_results: Optional[int] = 5):
        """Perform a web search with a query and return the result."""
        try:
            # Ensure num_results is an integer
            search_result = WebTools.serper_search(query=query, num_results=int(num_results))
            return search_result
        except Exception as e:
            raise Exception(f"Failed to perform web Serper search: {str(e)}")


### Serper  Shopping Search

In [6]:
from taskflowai import WebTools
from typing import Optional

class SerperShoppingSearch:
    @staticmethod
    def search_web(query: str, num_results: Optional[int] = 5):
        """Perform a shopping-related web search using the Serper API.

        Args:
            query (str): The search query.
            num_results (Optional[int]): Number of results to fetch (default: 5).

        Returns:
            dict: The search results from the API.

        Raises:
            RuntimeError: If the search fails.
        """
        try:
            # Ensure num_results is an integer and has a valid default
            num_results = int(num_results) if num_results is not None else 5

            # Perform the search
            search_result = WebTools.serper_search(
                query=query,
                search_type="shopping",
                num_results=num_results
            )
            return search_result
        except Exception as e:
            raise RuntimeError(f"Failed to perform web Serper search: {str(e)}")


### Web Research Agent

In [7]:
from taskflowai import Agent

class WebResearchAgent:
    @staticmethod
    def initialize_web_research_agent():
        """
        Initializes and returns the Crop or Plant Web Research Agent.
        """
        try:
            web_research_agent = Agent(
                role="Crop and Plant Research Agent",
                goal="Collect key details about a crop or plant, including classification, uses, growth conditions, and images.",
                attributes="Accurate, structured, and data-driven.",
                llm=LoadModel.load_openai_model(),
                tools=[
                    WikiArticles.fetch_articles,
                    WikiImages.search_images,
                    SerperSearch.search_web

                ],
            )

            return web_research_agent

        except Exception as e:
            raise Exception(f"Failed to initialize Web Research Agent: {str(e)}")


### Price Fetching Agent

In [8]:
from taskflowai import Agent

class PriceFetchingAgent:
    @staticmethod
    def initialize_price_fetching_agent(query: str):
        """
        Initializes and returns the Crop or Plant Price Research Agent.

        Args:
            query (str): The plant name or keyword for price research.

        Returns:
            Agent: A price research agent instance.
        """
        try:
            price_fetching_agent = Agent(
                role="Price Research Agent",
                goal="Find and compare the best prices for crops and plants across different markets.",
                attributes="Cost-conscious, data-driven, and focused on accurate price comparisons.",
                llm=LoadModel.load_openai_model(),
                tools=[SerperShoppingSearch.search_web],
            )

            return price_fetching_agent

        except Exception as e:
            raise Exception(f"Error initializing Price Research Agent: {str(e)}")


### reseach overall web task

In [10]:
from taskflowai import Task

def research_overall_web(plant_name: str):
    """
    Conducts an in-depth web research on the given plant and gathers comprehensive details.

    Goal:
    - To collect and summarize high-quality information about a plant species from trusted web sources.

    Attributes:
    - Scientific Name: The botanical classification of the plant.
    - History & Origin: Where the plant originates from, its historical significance, and domestication details.
    - Major Growing Regions: Countries and climates where the plant thrives.
    - Uses & Benefits: Agricultural, medicinal, nutritional, and industrial applications.
    - Growth Conditions: Soil type, temperature, humidity, and watering requirements.
    - Common Pests & Diseases: Typical threats and prevention strategies.
    - Cultural & Economic Significance: Traditional and commercial importance in different regions.
    - Relevant Images: Well-labeled images of the plant formatted as ![Description](https://full-image-url).

    Returns:
    - A structured research task that gathers this information using an AI-powered web research agent.

    Raises:
    - Exception: If the research process encounters an error.
    """

    try:
        # Initialize the research agent
        agent = WebResearchAgent.initialize_web_research_agent()

        # Create and execute the research task
        task = Task.create(
            agent=agent,
            context=f"Plant Name: {plant_name}",
            instruction=(
                f"Conduct thorough research on {plant_name} and provide structured information on:\n"
                f"- **Scientific Name**: Botanical classification.\n"
                f"- **History & Origin**: Historical significance and domestication.\n"
                f"- **Major Growing Regions**: Suitable climates and geographical distribution.\n"
                f"- **Uses & Benefits**: Applications in agriculture, medicine, and industry.\n"
                f"- **Growth Conditions**: Soil type, temperature, humidity, and watering needs.\n"
                f"- **Common Pests & Diseases**: Common threats and mitigation strategies.\n"
                f"- **Cultural & Economic Significance**: Importance in different regions.\n"
                f"- **Relevant Images**: Provide high-quality images formatted as ![Description](https://full-image-url).\n"
            ),
        )

        return task

    except Exception as e:
        raise Exception(f"Error in research_overall_web for {plant_name}: {str(e)}")


### Research health task

In [11]:
from taskflowai import Task

def research_health(plant_name: str):
    """Research health-related information about the plant.

    Args:
        plant_name (str): The name of the plant to research.

    Returns:
        Task: The research task instance containing health-related insights.
    """
    try:
        agent = WebResearchAgent.initialize_web_research_agent()

        task = Task.create(
            agent=agent,
            context=f"Plant Name: {plant_name}",
            instruction=(
                f"Research the health aspects of {plant_name}, including:\n"
                f"- Key health benefits and medicinal uses.\n"
                f"- Potential risks, side effects, and toxicity levels.\n"
                f"- Nutritional value (vitamins, minerals, and compounds).\n"
                f"- Common medical or traditional remedies using {plant_name}.\n"
                f"- Scientific studies or expert insights supporting health claims.\n"
                f"Provide structured and well-referenced information."
            ),
        )

        return task

    except Exception as e:
        raise Exception(f"Error in research_health for {plant_name}: {str(e)}")


### Research Season task

In [12]:
from taskflowai import Task

def research_season(plant_name: str):
    """Research seasonal growth and farming details of the plant.

    Args:
        plant_name (str): The name of the plant to research.

    Returns:
        Task: The research task instance with seasonal farming insights.
    """
    try:
        agent = WebResearchAgent.initialize_web_research_agent()

        task = Task.create(
            agent=agent,
            context=f"Plant Name: {plant_name}",
            instruction=(
                f"Research the optimal growth conditions for {plant_name}, covering:\n"
                f"- Ideal planting and harvesting seasons.\n"
                f"- Climate, temperature, and humidity requirements.\n"
                f"- Best soil types, nutrients, and fertilizers.\n"
                f"- Farming practices for maximizing yield.\n"
                f"- Alternative uses or storage methods for off-seasons.\n"
                f"Provide reliable agricultural data and expert recommendations."
            ),
        )

        return task

    except Exception as e:
        raise Exception(f"Error in research_season for {plant_name}: {str(e)}")


### price research task

In [13]:
from taskflowai import Task

def research_price(plant_name: str):
    """Research market price details of the plant.

    Args:
        plant_name (str): The name of the plant to research.

    Returns:
        Task: The research task instance with market pricing insights.
    """
    try:
        agent = PriceFetchingAgent.initialize_price_fetching_agent(query=plant_name)

        task = Task.create(
            agent=agent,
            context=f"Plant Name: {plant_name}",
            instruction=(
                f"Gather market price information for {plant_name}, focusing on:\n"
                f"- Price comparisons across online and offline markets.\n"
                f"- Cost per kilogram vs. per pound for different quality grades.\n"
                f"- Price variations based on country, state, or region (prioritize user location).\n"
                f"- Factors affecting pricing (seasonality, demand, supply, and quality).\n"
                f"Provide accurate, up-to-date data from reliable sources."
            ),
        )

        return task

    except Exception as e:
        raise Exception(f"Error in research_price for {plant_name}: {str(e)}")


## Execute Research and Report - flow 1

In [14]:
def execute_research_and_report(plant_names):
    """Executes research tasks and generates a structured report for each plant.

    Args:
        plant_names (list): A list of plant names to research.

    Returns:
        dict: Research results categorized by plant name.
    """
    research_results = {}
    try:
        if not plant_names:
            return research_results

        for plant_name in plant_names:
            research_results[plant_name] = {
                "general": research_overall_web(plant_name),
                "health": research_health(plant_name),
                "season": research_season(plant_name),
                "price": research_price(plant_name),
            }

        return research_results

    except Exception as e:
        raise Exception(f"Error in execute_research_and_report: {str(e)}")


## Generate Summarized Report

In [15]:
def generate_summarized_report(research_results):
    """Generates a structured and easy-to-read summarized report from research results.

    Args:
        research_results (dict): A dictionary containing research data for each plant.

    Returns:
        str: A formatted summary report of the research.
    """
    try:
        if not research_results:
            return "No research results to summarize."

        summarized_report = "🌿 **Plant Research Summary** 🌿\n\n"

        for plant, results in research_results.items():
            summarized_report += f"🌱 **{plant.capitalize()}**\n"
            summarized_report += f"📌 **General Information:** {results.get('general', 'No data available')}\n"
            summarized_report += f"🩺 **Health Benefits & Risks:** {results.get('health', 'No data available')}\n"
            summarized_report += f"🌤 **Growing Season & Conditions:** {results.get('season', 'No data available')}\n"
            summarized_report += f"💰 **Market Prices & Trends:** {results.get('price', 'No data available')}\n"
            summarized_report += "-" * 50 + "\n\n"

        return summarized_report

    except Exception as e:
        raise Exception(f"Error in generate_summarized_report: {str(e)}")


### Endpoint

In [17]:
if __name__ == "__main__":
    plant_name = input("Enter the plant name(s) (comma-separated if multiple): ").strip()

    # Convert input into a list of plant names
    plant_names = [name.strip() for name in plant_name.split(",")]

    try:
        # Execute research and collect results
        research_results = execute_research_and_report(plant_names)

        # Print research results one by one as they are processed
        for plant, results in research_results.items():
            print(f"\n--- Research Results for {plant} ---")
            print("General Information:\n", results.get("general", "No data available"))
            print("\nHealth Research:\n", results.get("health", "No data available"))
            print("\nSeasonal Research:\n", results.get("season", "No data available"))
            print("\nPrice Research:\n", results.get("price", "No data available"))
            print("\n" + "-" * 40)  # Separator for clarity

        # Generate and print summarized report
        summarized_report = generate_summarized_report(research_results)
        print("\nFinal Summarized Report:\n")
        print(summarized_report)

    except Exception as e:
        print("\nAn error occurred:", str(e))


Enter the plant name(s) (comma-separated if multiple): avacado, tulsi
✔ Request completed
Tool Use: search_web
Parameters:
  query: avocado scientific name

Result: Organic Results:
1. Avocado - Wikipedia
   URL: https://en.wikipedia.org/wiki/Avocado
   Snippet: The avocado, alligator pear or avocado pear (Persea americana) is an evergreen tree in the laurel family (Lauraceae). It is native to the Americas

2. Persea americana - Plant Finder - Missouri Botanical Garden
   URL: https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid...

Tool Use: search_web
Parameters:
  query: avocado history and origin

Result: Organic Results:
1. History - Avocados From Mexico
   URL: https://avocadosfrommexico.com/avocados/history/
   Snippet: Researchers believe Puebla, located in South Central Mexico, to be the motherland of the avocado, where this strange and delicious fruit first flourished.

2. Avocado - Wikipedia
   URL: https://en.wikipedia.org/wiki/Avocado
   Snip